[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WarrenTseng/genai_tutorials/blob/main/MONAI/CXR/CXR.ipynb)

## Install Requirements

In [ ]:
!pip install monai fire 
!pip install --upgrade --no-cache-dir gdown 
!pip install transformers
!git clone https://github.com/Project-MONAI/GenerativeModels.git
%cd GenerativeModels/
!python setup.py install

## Check Installation

In [ ]:
import monai
import torch
import generative
import gdown
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

monai.__version__, generative.__version__

## Download Pre-trained Model

In [ ]:
%cd model-zoo/models/cxr_image_synthesis_latent_diffusion_model/
!mkdir models
%cd models

In [ ]:
import gdown
url = "https://drive.google.com/uc?export=download&id=1paDN1m-Q_Oy8d_BanPkRTi3RlNB_Sv_h"
output = "autoencoder.pth"
gdown.download(url, output, quiet=False)

url = "https://drive.google.com/uc?export=download&id=1CjcmiPu5_QWr-f7wDJsXrCCcVeczneGT"
output = "diffusion_model.pth"
gdown.download(url, output, quiet=False)
!ls

## Synthetic Data Generation

In [ ]:
%cd ..
!mkdir output

In [ ]:
!python -m monai.bundle run save_jpg --config_file configs/inference.json \
        --prompt "Big right-sided pleural effusion"

## Preview

In [ ]:
fname = [f for f in os.listdir('output') if 'ipynb' not in f][-1]
fname

In [ ]:
filepath = 'output/' + fname
img = plt.imread(filepath)

plt.imshow(img, cmap='gray')
plt.show()

## Generate Training Data

- Big right-sided pleural effusion

In [ ]:
url = 'https://drive.google.com/file/d/14Z0Vmv-8_mGSZXOrtw0Cqxnks0jC_I-1/view?usp=sharing'
output = 'inference_n.json'
gdown.download(url, output, quiet=False, fuzzy=True)
!mv inference_n.json configs/

In [ ]:
!rm -r output/*
!mkdir dataset

In [ ]:
!python -m monai.bundle run save_jpg --config_file configs/inference_n.json \
        --prompt "Big right-sided pleural effusion" \
        --n_imgs 50

In [ ]:
!mkdir dataset/1
!mv output/* dataset/1/

- Small right-sided pleural effusion

In [ ]:
!python -m monai.bundle run save_jpg --config_file configs/inference_n.json \
        --prompt "Small right-sided pleural effusion" \
        --n_imgs 50

In [ ]:
!mkdir dataset/2
!mv output/* dataset/2/

## Training

In [ ]:
%cd GenerativeModels/model-zoo/models/cxr_image_synthesis_latent_diffusion_model/
import gdown

url = 'https://drive.google.com/file/d/1_yJkKyzYwpiQUytoKhUjmtnJ3jxKdejO/view?usp=drive_link'
output = 'cxr_train.py'
gdown.download(url, output, quiet=False, fuzzy=True)

In [ ]:
from cxr_train import CXR_demo

max_epoch = 50
class1_path = 'dataset/1/'
class2_path = 'dataset/2/'
cxr_demo = CXR_demo()

In [ ]:
cxr_demo.train(class1_path, class2_path, max_epoch)

In [ ]:
cxr_demo.vis_result('models/best.pth')